In [1]:
import json
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from GoogleScraper import scrape_with_config, GoogleSearchError
from GoogleScraper.database import ScraperSearch, SERP, Link
import requests
import time

# 1. Configure Google Search

In [2]:
config = {
    'keyword': 'should site:reddit.com/r/AskReddit/',
    'num_pages_for_keyword': 100,
    'scrape_method': 'http',
    'do_caching': True,
    'output_filename': 'Results/google.json',
    'google_search_params': {'cr': 'countryUS'}
}

# 2. Get Google Results

In [3]:
try:
    search = scrape_with_config(config)
except GoogleSearchError as e:
    print(e)

# 3. Extract Links

In [4]:
links = []
for serp in search.serps:
    for link in serp.links:
        if link.link.startswith('https://www.reddit.com/r/AskReddit/'):
            links.append(link.link.strip())

# 4. Scrape Links

In [6]:
scrape = []
for link in links:
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    question_f = soup.find("a", { "class" , "title may-blank" })
    comments_f = soup.find("a", { "class" , "bylink comments may-blank" })
    
    if question_f is not None:
        question = question_f.text
        comments = -1
        if comments_f is not None:
            comments = comments_f.text
        
        scrape.append([link, question.strip(), comments])
    else:
        scrape.append([link, None, None])
        

KeyboardInterrupt: 

# 5. Output Results as CSV

In [6]:
scrapeDF = pd.DataFrame(scrape, columns=['Link', 'Question', 'Answers/Comments'])
scrapeDF.to_csv('Results/Reddit Scrape Results_Should' + time.strftime("%Y%m%d-%H%M%S") + '.csv', index=False)

# END